In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Capstone | Bangkit 2023/Sentiment Analysis/cleaned.csv')
df.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment
0,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Barep Pratama,Akhirnyaaaa bisa nulis review yang memuaskannn...,5,NaN
1,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Luqman Heryana,Cafe de paris\nTerletak di timur pantai parang...,5,NaN
2,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Catharine Winata,"masakannya enak tp kecil2 sesuai dg harganya😊,...",4,NaN
3,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Dewi Pustaka,"Ownernya baik banget, nginep disini pas Ramadh...",5,NaN
4,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Kiki Melaendo,"Tempatnya bagus bgt, makanan enak dan cocok bu...",5,NaN


# Check and fill the missing data

In [4]:
place = pd.read_csv('/content/drive/MyDrive/Capstone | Bangkit 2023/Review/Places.csv')
place.head()

,Place_ID,Name,FormattedPhone,FormattedAddress,Latitude,Longitude,OverallRating,PriceLevel,Website,UserRatingTotal,...,ServesWine,ServesVegetarianFood,WheelchairAccessibleEntrance,Halal,StreetAddress,District,City,Regency,Province,PostalNumber
0,ChIJ8b3du_dZei4R5Csj9BqdzsM,LOKO CAFE STASIUN TUGU,0812-2009-6492,"Sosromenduran, Gedong Tengen, Kota Yogyakarta,...",-7.789311,110.363483,4.1,NaN,https://instagram.com/lokocafe?igshid=OTJlNzQ0...,12.0,...,NaN,NaN,NaN,0,NaN,NaN,Kota Yogyakarta,NaN,Daerah Istimewa,Yogyakarta
1,ChIJ-1OO3xhZei4RpiUw_jIlPBQ,TUGU LOR,(0274) 515268,"Cokrodiningratan, Kec. Jetis, Kota Yogyakarta,...",-7.774089,110.368301,4.5,2.0,NaN,1644.0,...,0.0,NaN,0.0,1,NaN,Kec. Jetis,Kota Yogyakarta,NaN,Daerah Istimewa Yogyakarta,55233
2,ChIJAbDRQ7pZei4Rm8oXtJg74k8,Ramu Nostalgia - Tugu Yogyakarta,0851-5692-3734,"Jl. A.M. Sangaji No.2, Cokrodiningratan, Kec. ...",-7.782492,110.367303,5.0,NaN,http://www.ramunostalgia.com/,17.0,...,0.0,NaN,NaN,1,Jl. A.M. Sangaji No.2,Kec. Jetis,Kota Yogyakarta,NaN,Daerah Istimewa Yogyakarta,55233
3,ChIJWW-U4vlZei4RHX6ILzVTvcA,Kedai Kolega Tugu,0821-3750-7055,"Jl. P. Mangkubumi No.21, Gowongan, Kec. Jetis,...",-7.787412,110.366382,4.6,NaN,NaN,59.0,...,0.0,NaN,NaN,1,Jl. P. Mangkubumi No.21,Kec. Jetis,Kota Yogyakarta,NaN,Daerah Istimewa Yogyakarta,55232
4,ChIJq6pHmTBYei4RlTjahkZgUlY,Wedangan,NaN,"Jl. A.M. Sangaji No.9, Cokrodiningratan, Kec. ...",-7.782092,110.367058,5.0,NaN,NaN,10.0,...,0.0,NaN,NaN,1,Jl. A.M. Sangaji No.9,Kec. Jetis,Kota Yogyakarta,NaN,Daerah Istimewa Yogyakarta,55233


In [5]:
place[place['Name'].str.contains("Kopi PM")]

,Place_ID,Name,FormattedPhone,FormattedAddress,Latitude,Longitude,OverallRating,PriceLevel,Website,UserRatingTotal,...,ServesWine,ServesVegetarianFood,WheelchairAccessibleEntrance,Halal,StreetAddress,District,City,Regency,Province,PostalNumber
23,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꧋ꦏꦺꦴꦥꦶ꧇ꦥ꧈ꦩ꧈꧇꧉,0878-3965-2668,"Alun alun utara, Kecamatan Kraton, Kota Yogyak...",-7.804102,110.365875,5.0,NaN,NaN,14.0,...,NaN,NaN,NaN,0,NaN,Kecamatan Kraton,Kota Yogyakarta,NaN,Daerah Istimewa Yogyakarta,55131


In [6]:
df.isna().sum()

Place_ID        25
Place_Name       0
User             0
Review           0
Star             0
Sentiment     5566
dtype: int64

In [7]:
df[df.Place_ID.isna()]

,Place_ID,Place_Name,User,Review,Star,Sentiment
2237,NaN,Kopi PM ꦏꦥꦥꦩ,Abdul Aziz,Teman2 muslim harap hati2 karena ada minuman b...,4,NaN
2238,NaN,Kopi PM ꦏꦥꦥꦩ,Agung Aryanto,Coffeeshop yang cukup cakep di daerah selatan ...,5,NaN
2239,NaN,Kopi PM ꦏꦥꦥꦩ,deandra fariska,Pertama kali kesini langsung jatuh cinta sama ...,5,NaN
2240,NaN,Kopi PM ꦏꦥꦥꦩ,Sukarno,Salah satu tempat nongski asyik di selatan Jog...,5,NaN
2241,NaN,Kopi PM ꦏꦥꦥꦩ,Reza Horisman,Lokasinya persis dibelokan pinggir jalan by pa...,5,NaN
2242,NaN,Kopi PM ꦏꦥꦥꦩ,Dhimas B,Lokasi cafe ini ada diringroad selatan. Pas be...,5,NaN
2243,NaN,Kopi PM ꦏꦥꦥꦩ,Nadiah Mumtaz,"Cafe dengan tema serba putih, earthy, artsy, b...",5,NaN
2244,NaN,Kopi PM ꦏꦥꦥꦩ,Beenta,Tempatnya asyik buat nongkrong. Teh Leci nya k...,4,NaN
2245,NaN,Kopi PM ꦏꦥꦥꦩ,bambang irawan,"Lokasi beneran pas pojok, jd kudu dijalur lamb...",3,NaN
2246,NaN,Kopi PM ꦏꦥꦥꦩ,Ludovica Elisa,Awalnya gatau kl ada kopisyopp seenak dan seny...,5,NaN


In [8]:
df['Place_ID'] = df.Place_ID.fillna('ChIJu3TwQcRZei4RNxcUTwE6AJI')

In [9]:
df[df.Place_ID.isna()]

,Place_ID,Place_Name,User,Review,Star,Sentiment


In [10]:
df[df['Place_Name'].str.contains("Kopi PM")]

,Place_ID,Place_Name,User,Review,Star,Sentiment
2237,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,Abdul Aziz,Teman2 muslim harap hati2 karena ada minuman b...,4,NaN
2238,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,Agung Aryanto,Coffeeshop yang cukup cakep di daerah selatan ...,5,NaN
2239,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,deandra fariska,Pertama kali kesini langsung jatuh cinta sama ...,5,NaN
2240,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,Sukarno,Salah satu tempat nongski asyik di selatan Jog...,5,NaN
2241,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,Reza Horisman,Lokasinya persis dibelokan pinggir jalan by pa...,5,NaN
2242,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,Dhimas B,Lokasi cafe ini ada diringroad selatan. Pas be...,5,NaN
2243,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,Nadiah Mumtaz,"Cafe dengan tema serba putih, earthy, artsy, b...",5,NaN
2244,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,Beenta,Tempatnya asyik buat nongkrong. Teh Leci nya k...,4,NaN
2245,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,bambang irawan,"Lokasi beneran pas pojok, jd kudu dijalur lamb...",3,NaN
2246,ChIJu3TwQcRZei4RNxcUTwE6AJI,Kopi PM ꦏꦥꦥꦩ,Ludovica Elisa,Awalnya gatau kl ada kopisyopp seenak dan seny...,5,NaN


In [11]:
df.isna().sum()

Place_ID         0
Place_Name       0
User             0
Review           0
Star             0
Sentiment     5566
dtype: int64

## Clean the review

In [12]:
import string 
import re 

In [13]:
# remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

# lowercase string
def lowercase_string(text):
    return text.lower()

# Remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

# Remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

# Remove multiple whitespace into single whitespace -- leading & trailing whitespace
def remove_whitespace_multiple(text):
    return ' '.join(text.split())

#Correcting the repeating characters in the end of the word
def correct_repeating_characters(text):
    # Remove repeating characters at the end of words
    corrected_text = re.sub(r'(\w)(\1{2,})\b', r'\1', text)
    
    # Remove repeating spaces
    corrected_text = re.sub(r'\s+', ' ', corrected_text)
    
    return corrected_text

# remove special characters
def remove_special_characters(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text

def remove_emoticons(text):
    emoticon_pattern = re.compile(
        u"[\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002600-\U000026FF"  # miscellaneous symbols
        u"\U00002700-\U000027BF"  # dingbats
        u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
        "]+"
    )
    cleaned_text = re.sub(emoticon_pattern, '', text)
    return cleaned_text

In [14]:
reviews = df.Review.astype(str)

In [15]:
reviews

0       Akhirnyaaaa bisa nulis review yang memuaskannn...
1       Cafe de paris\nTerletak di timur pantai parang...
2       masakannya enak tp kecil2 sesuai dg harganya😊,...
3       Ownernya baik banget, nginep disini pas Ramadh...
4       Tempatnya bagus bgt, makanan enak dan cocok bu...
                              ...                        
5561    Ya ternyata dibalik belum sempurna ada hikmah....
5562    Salah satu dari dua plengkung yang masih utuh ...
5563    Katanya ada grosir kembang api nama tokonya ap...
5564                  vibe sejarahnya dpt banget , klasik
5565    Jalan masuk ke pusat kuliner gudeg jogja, ting...
Name: Review, Length: 5566, dtype: object

In [16]:
cleaned_reviews = reviews.apply(lowercase_string)\
                  .apply(remove_number) \
                  .apply(remove_emoticons) \
                  .apply(remove_punctuation) \
                  .apply(remove_single_char) \
                  .apply(remove_whitespace_multiple) \
                  .apply(correct_repeating_characters) \
                  .apply(remove_special_characters) 
                  

In [17]:
cleaned_reviews

0       akhirnya bisa nulis review yang memuaskan lagi...
1       cafe de paris terletak di timur pantai parangt...
2       masakannya enak tp kecil sesuai dg harganya pi...
3       ownernya baik banget nginep disini pas ramadha...
4       tempatnya bagus bgt makanan enak dan cocok bua...
                              ...                        
5561    ya ternyata dibalik belum sempurna ada hikmah ...
5562    salah satu dari dua plengkung yang masih utuh ...
5563    katanya ada grosir kembang api nama tokonya ap...
5564                    vibe sejarahnya dpt banget klasik
5565    jalan masuk ke pusat kuliner gudeg jogja tingg...
Name: Review, Length: 5566, dtype: object

## Normalization (Correct the slang word)

In [18]:
slang_df = pd.read_csv('/content/drive/MyDrive/Capstone | Bangkit 2023/Sentiment Analysis/new_kamusalay.csv', header = None, encoding='latin-1')
slang_df.head()

,0,1
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [19]:
slang_dict = dict(zip(slang_df[0], slang_df[1]))

In [20]:
def correct_slang_words(text, slang_dict):
    words = text.split()
    corrected_words = []
    
    for word in words:
        if word in slang_dict:
            corrected_word = slang_dict[word]
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)
    
    corrected_text = ' '.join(corrected_words)
    return corrected_text

In [21]:
cleaned_reviews = cleaned_reviews.apply(lambda x: correct_slang_words(x, slang_dict))

In [22]:
cleaned_reviews

0       akhirnya bisa menulis review yang memuaskan la...
1       cafe deh paris terletak di timur pantai parang...
2       masakannya enak tetapi kecil sesuai dengan har...
3       ownernya baik banget menginap di sini pas rama...
4       tempatnya bagus banget makanan enak dan cocok ...
                              ...                        
5561    ya ternyata dibalik belum sempurna ada hikmah ...
5562    salah satu dari dua plengkung yang masih utuh ...
5563    katanya ada grosir kembang api nama tokonya ap...
5564                  vibe sejarahnya dapat banget klasik
5565    jalan masuk ke pusat kuliner gudeg yoyakarta t...
Name: Review, Length: 5566, dtype: object

## Remove Stopwords

In [23]:
from nltk.corpus import stopwords

In [24]:
stopwords = pd.read_csv('/content/drive/MyDrive/Capstone | Bangkit 2023/Sentiment Analysis/stopwordbahasa.csv', header = None)
stopwords.head()

,0
0,ada
1,adalah
2,adanya
3,adapun
4,agak


In [25]:
stopwords[stopwords.index.isin([46,50, 66,232,351,357, 371,725,726,727,728])]

,0
46,baik
50,banyak
66,belum
232,dua
351,lama
357,lima
371,masalah
725,tidak
726,tidakkah
727,tidaklah


In [26]:
stopwords_fix = stopwords[stopwords.index.isin([46,50, 66,351, 371,725,726,727])]
stopwords_fix.columns = ['stopword']

In [27]:
new_stopwords = ['yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 
                  'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                  'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                  'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                  'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                  'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                  '&amp', 'yah', 'nya', 'untuk', 'e']

# Convert the list to a DataFrame
new_stopwords_df = pd.DataFrame(new_stopwords, columns=['stopword'])

# Concatenate the new DataFrame with the original DataFrame
stopwords = pd.concat([stopwords_fix, new_stopwords_df], ignore_index=True)

In [28]:
stopwords.head()

,stopword
0,baik
1,banyak
2,belum
3,lama
4,masalah


In [29]:
stopwords.tail()

,stopword
44,rt
45,&amp
46,yah
47,nya
48,untuk


In [30]:
def remove_stopwords(text, stopwords):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [31]:
cleaned_reviews = cleaned_reviews.apply(lambda x: remove_stopwords(x, stopwords['stopword']))

In [32]:
cleaned_reviews

0       akhirnya bisa menulis review yang memuaskan la...
1       cafe deh paris terletak di timur pantai parang...
2       masakannya enak tetapi kecil sesuai dengan har...
3       ownernya baik banget menginap di sini pas rama...
4       tempatnya bagus banget makanan enak dan cocok ...
                              ...                        
5561    ya ternyata dibalik belum sempurna ada hikmah ...
5562    salah satu dari dua plengkung yang masih utuh ...
5563    katanya ada grosir kembang api nama tokonya ap...
5564                  vibe sejarahnya dapat banget klasik
5565    jalan masuk ke pusat kuliner gudeg yoyakarta t...
Name: Review, Length: 5566, dtype: object

In [33]:
cleaned_reviews.to_list()

['akhirnya bisa menulis review yang memuaskan lagi enak banget ikan bakarnyaa makanan semua oke sentuhan bumbunya lain dari pada yang lain ibu bapak lupa belum kenalan super ramah couple goals bisa haha hihi sampai menua bersama sehat selalu bapak ibu janji deh akan balik lagi suka banget sama tempat dan masakan ibunya bule saja makan di sini wisatawan lokal jangan mau kalah dongggjelajahi terus wisata indonesia terima kasih bonus tunanya busempat galau antara kakap atau tuna akhirnya malah bisa merasakan keduanyaa',
 'cafe deh paris terletak di timur pantai parangtritis dan akses dengan jalan kaki dari parkiran terakhir makanannya cukup banyak pilihan mulai dari seafood sayur dan minuman suasannya nyaman dan santai dengan banyak pilihan tempat duduk ayunan tempat tidur gantung serta gazebo konsep nya cafe pantai yang asyik harga nya murah untuk di tempat wisata worth it',
 'masakannya enak tetapi kecil sesuai dengan harganya pisang gorengnya mantap sambelnya mantap',
 'ownernya baik b

## Remove -nya

In [34]:
def remove_nya(text):
    cleaned_text = re.sub(r'(\w+)nya\b', r'\1', text)
    return cleaned_text

In [35]:
cleaned_reviews = cleaned_reviews.apply(remove_nya)

In [36]:
cleaned_reviews.to_list()

['akhir bisa menulis review yang memuaskan lagi enak banget ikan bakarnyaa makanan semua oke sentuhan bumbu lain dari pada yang lain ibu bapak lupa belum kenalan super ramah couple goals bisa haha hihi sampai menua bersama sehat selalu bapak ibu janji deh akan balik lagi suka banget sama tempat dan masakan ibu bule saja makan di sini wisatawan lokal jangan mau kalah dongggjelajahi terus wisata indonesia terima kasih bonus tuna busempat galau antara kakap atau tuna akhir malah bisa merasakan keduanyaa',
 'cafe deh paris terletak di timur pantai parangtritis dan akses dengan jalan kaki dari parkiran terakhir makanan cukup banyak pilihan mulai dari seafood sayur dan minuman suasan nyaman dan santai dengan banyak pilihan tempat duduk ayunan tempat tidur gantung serta gazebo konsep nya cafe pantai yang asyik harga nya murah untuk di tempat wisata worth it',
 'masakan enak tetapi kecil sesuai dengan harga pisang goreng mantap sambel mantap',
 'owner baik banget menginap di sini pas ramadan p

## Transfer Learning - Indobert

In [39]:
pip install tensorflow transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.6 MB/s eta 0:00:00


### Transfer Learning

In [40]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, TFAutoModel
import tensorflow as tf

In [41]:
model1 = 'w11wo/indonesian-roberta-base-sentiment-classifier'

### data

In [42]:
columns = ['text', 'label']
train = pd.read_table('/content/drive/MyDrive/Capstone | Bangkit 2023/Sentiment Analysis/traintest_data/train_data_restaurant.tsv', header = None, names = columns)
train.head()

,text,label
0,Saya makan di sini dengan menggunakan voucher ...,negative
1,Mas indosat buat main game enemy has been slai...,negative
2,Trus sy percuma dong update no hp terbaru ke c...,negative
3,jika kita pilih dia maka akan ada lebih dari 3...,negative
4,Dikarenakan adik saya suka dengan restoran ini...,negative


In [43]:
test = pd.read_table('/content/drive/MyDrive/Capstone | Bangkit 2023/Sentiment Analysis/traintest_data/test_data_restaurant.tsv', header = None, names = columns)
test.head()

,text,label
0,"Tempatnya sih menarik, mudah dijangkau dari se...",negative
1,lokasi strategis penasaran karena daerah situ ...,negative
2,"Sesuai namanya, restoran ini unik dalam menyaj...",negative
3,Saya datang ketika petang dan hujan deras temp...,negative
4,"Kalao mau ke sana jangan pas malem sih, soalny...",negative


In [44]:
# Define the mapping dictionary
mapping_dict = {'negative': 0, 'positive': 1}

# Map the values using the dictionary
train['label'] = train['label'].map(mapping_dict)
test['label'] = test['label'].map(mapping_dict)

In [45]:
train.head()

,text,label
0,Saya makan di sini dengan menggunakan voucher ...,0
1,Mas indosat buat main game enemy has been slai...,0
2,Trus sy percuma dong update no hp terbaru ke c...,0
3,jika kita pilih dia maka akan ada lebih dari 3...,0
4,Dikarenakan adik saya suka dengan restoran ini...,0


In [46]:
def clean_data(reviews):
  reviews = reviews.apply(lowercase_string)\
                  .apply(remove_number) \
                  .apply(remove_emoticons) \
                  .apply(remove_punctuation) \
                  .apply(remove_single_char) \
                  .apply(remove_whitespace_multiple) \
                  .apply(correct_repeating_characters) \
                  .apply(remove_special_characters) 

  reviews = reviews.apply(lambda x: correct_slang_words(x, slang_dict))

  reviews = reviews.apply(lambda x: remove_stopwords(x, stopwords['stopword']))

  reviews = reviews.apply(remove_nya)

  return reviews

In [47]:
train['text_cleaned'] = clean_data(train['text'])

In [48]:
train.head()

,text,label,text_cleaned
0,Saya makan di sini dengan menggunakan voucher ...,0,saya makan di sini dengan menggunakan kupon di...
1,Mas indosat buat main game enemy has been slai...,0,mas indosat buat main game enemy has been sela...
2,Trus sy percuma dong update no hp terbaru ke c...,0,terus saya percuma dong pembaruan no handphone...
3,jika kita pilih dia maka akan ada lebih dari 3...,0,jika kita pilih dia maka akan ada lebih dari k...
4,Dikarenakan adik saya suka dengan restoran ini...,0,dikarenakan adik saya suka dengan restoran ini...


In [49]:
test['text_cleaned'] = clean_data(test['text'])

In [50]:
class config:
    MAX_LEN = 36
    LOWER_CASE = True
    RANDOM_STATE = 12
    TEST_SIZE = 0.2
    NUM_LABELS = 1
    BATCH_SIZE = 128
    LEARNING_RATE = 5e-5
    EPOCHS = 10

In [51]:
tokenizer = RobertaTokenizer.from_pretrained(model1)
x_train = tokenizer(
    text = train["text_cleaned"].tolist(),
    add_special_tokens=True,
    max_length=config.MAX_LEN,
    truncation=True,
    padding=True,
    return_tensors="tf",
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

x_test = tokenizer(
    text = test["text_cleaned"].tolist(),
    add_special_tokens=True,
    max_length=config.MAX_LEN,
    truncation=True,
    padding=True,
    return_tensors="tf",
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

In [52]:
roberta_base = TFAutoModel.from_pretrained(model1)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
MAX_LEN = 36
LOWER_CASE = True
RANDOM_STATE = 144
TEST_SIZE = 0.2
NUM_LABELS = 1
BATCH_SIZE = 128
LEARNING_RATE = 5e-3
EPOCHS = 5

In [54]:
input_ids = tf.keras.Input(shape=(MAX_LEN), dtype=tf.int32, name="input_ids")
input_mask = tf.keras.Input(shape=(MAX_LEN), dtype=tf.int32, name="attention_mask")

embeddings = roberta_base(input_ids, attention_mask=input_mask)[1]

x = tf.keras.layers.Dropout(0.3)(embeddings)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation="relu")(x)
output = tf.keras.layers.Dense(cNUM_LABELS, activation="sigmoid")(x)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=output)

In [55]:
model.layers[2].trainable = True
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 36)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 36)]         0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 36,                                            

In [56]:
optimizer = tf.keras.optimizers.Adam(learning_rate=config.LEARNING_RATE)

In [58]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=optimizer, metrics=["accuracy"])
history = model.fit(x={"input_ids": x_train["input_ids"],
                       "attention_mask": x_train["attention_mask"]},
                    y=train["label"],
                    epochs=EPOCHS,
                    validation_split=0.2,
                    batch_size=30)

Epoch 1/5
12/12 [==============================] - 724s 56s/step - loss: 0.4153 - accuracy: 0.8251 - val_loss: 0.2452 - val_accuracy: 0.8904
Epoch 2/5
12/12 [==============================] - 658s 53s/step - loss: 0.2427 - accuracy: 0.9115 - val_loss: 0.0924 - val_accuracy: 0.9607
Epoch 3/5
12/12 [==============================] - 666s 54s/step - loss: 0.1790 - accuracy: 0.9284 - val_loss: 0.2196 - val_accuracy: 0.9129
Epoch 4/5
12/12 [==============================] - 641s 54s/step - loss: 0.1024 - accuracy: 0.9712 - val_loss: 0.2789 - val_accuracy: 0.8989
Epoch 5/5
12/12 [==============================] - 640s 54s/step - loss: 0.0609 - accuracy: 0.9853 - val_loss: 0.3568 - val_accuracy: 0.8736


In [116]:
model.save('/content/drive/MyDrive/Capstone | Bangkit 2023/Sentiment Analysis/my_model')

In [117]:
model.save('/content/drive/MyDrive/Capstone | Bangkit 2023/Sentiment Analysis/my_model.h5')

In [59]:
reviews_test = tokenizer(
    text = cleaned_reviews.tolist(),
    add_special_tokens=True,
    max_length=config.MAX_LEN,
    truncation=True,
    padding=True,
    return_tensors="tf",
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

In [60]:
preds = model.predict({"input_ids": reviews_test["input_ids"], "attention_mask": reviews_test["attention_mask"]})
class_preds = np.where(preds > 0.5, 1, 0)
print(preds.shape, class_preds.shape)

174/174 [==============================] - 680s 4s/step
(5566, 1) (5566, 1)


In [61]:
class_preds

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [65]:
len(preds[preds > 0.99])

3633

In [66]:
preds

array([[0.99800575],
       [0.99801123],
       [0.99882245],
       ...,
       [0.05326146],
       [0.00900123],
       [0.99154204]], dtype=float32)

In [67]:
df['Sentiment'] = preds
df.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment
0,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Barep Pratama,Akhirnyaaaa bisa nulis review yang memuaskannn...,5,0.998006
1,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Luqman Heryana,Cafe de paris\nTerletak di timur pantai parang...,5,0.998011
2,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Catharine Winata,"masakannya enak tp kecil2 sesuai dg harganya😊,...",4,0.998822
3,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Dewi Pustaka,"Ownernya baik banget, nginep disini pas Ramadh...",5,0.997596
4,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Kiki Melaendo,"Tempatnya bagus bgt, makanan enak dan cocok bu...",5,0.998603


## Review with Positive Sentiment but less than 3 stars

In [73]:
pos_sentiment_less_3_star = df.query('Sentiment > 0.5 and Star < 3')
pos_sentiment_less_3_star.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment
175,ChIJc_LsfTBYei4R2ZbyfvHGvYw,Angkringan Kopi Joss Tugu Mbak lina,yenanda dwiyan,Makanannya rasanya B banget dan kurang cocok g...,1,0.982182
200,ChIJk5mlx3Dkei4R6qbkrzjBp5w,Angkringan Kopi Joss,yenanda dwiyan,Makanannya rasanya B banget dan kurang cocok g...,1,0.982182
441,ChIJA9fPd8Tlei4RnqDGJ9Jljro,Bakul jamu Bu Siti,M dalili Rohmatulloh,enak,2,0.834173
1103,ChIJT4C2kUZZei4RDc-3-pd3dsY,Don gO Coffee,Sophia 09,pisang gorengnya dikit pol kek 2 rb an\ntempat...,1,0.985095
2103,ChIJHcoTb4BZei4RxA3ibBrmUA8,Kopi Hoja Mie & Yu,Halaman Belakang,Makanan nya enak bener2 pas di lidah saya kasi...,1,0.971928


In [74]:
pos_sentiment_less_3_star.shape

(26, 6)

In [75]:
pos_sentiment_less_3_star['Review'].tolist()

['Makanannya rasanya B banget dan kurang cocok gak bisa bakar lagi, view doangbagus',
 'Makanannya rasanya B banget dan kurang cocok gak bisa bakar lagi, view doangbagus',
 'enak',
 'pisang gorengnya dikit pol kek 2 rb an\ntempat e mayan si sepi cocok buat intropet',
 'Makanan nya enak bener2 pas di lidah saya kasih bintang 5 tapi pelayanan nya tidak. Pelayan kurang membantu apalagi pemesanan melalui barcode pelanggan yg baru pertama kali datang binggung dan tdk dibantu sama sekali lalu meja yg sudah di pakai pun setelah dibayar dan di tempati orang lain bukannya membantu untuk mendapatkan meja itu kembali justru malah saya suruh pindah heheh mungkinh attitude nya bagi saya kurang ya karna saya yg bayar duluan tp pas di tempati orang lain pelayannya ga membantu sama sekali jujur kecewa mungkin diperbaiki lagi🙏🙏🙏',
 'Berkunjung ke Yogyakarta, biasanya saya selalu menyempatkan diri ke loco cafe. Karena jujur ada Live Music dan suasananya nostalgia sekali.\n\nTapi maaf untuk makanannya sa

**Some reviews reveal that many customers are dissatisfied with the service, taste of the food, or the price of the restaurant. However, there are also quite a few reviews that highlight the advantages of the restaurant.**

## Review with Positive Sentiment and 3 stars

In [79]:
pos_sentiment_3_star = df.query('Sentiment > 0.5 and Star == 3')
pos_sentiment_3_star.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment
12,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Riza Pahlevi,Tempatnya cukup asyik buat duduk2 santai menik...,3,0.510126
155,ChIJc_LsfTBYei4R2ZbyfvHGvYw,Angkringan Kopi Joss Tugu Mbak lina,Rahajeng Marlitasari Putri,Lokasi yang pas dan view yang kita dapetin itu...,3,0.997783
160,ChIJc_LsfTBYei4R2ZbyfvHGvYw,Angkringan Kopi Joss Tugu Mbak lina,kaktus hijau,Tempat enak langsung view tugu jogja kalo male...,3,0.998638
180,ChIJk5mlx3Dkei4R6qbkrzjBp5w,Angkringan Kopi Joss,Rahajeng Marlitasari Putri,Lokasi yang pas dan view yang kita dapetin itu...,3,0.997783
185,ChIJk5mlx3Dkei4R6qbkrzjBp5w,Angkringan Kopi Joss,kaktus hijau,Tempat enak langsung view tugu jogja kalo male...,3,0.998638


In [80]:
pos_sentiment_3_star.shape

(157, 6)

In [81]:
pos_sentiment_3_star.Review.tolist()

['Tempatnya cukup asyik buat duduk2 santai menikmati Pantai Parangtritis, sayang harganya cukup mahal, ya walaupun harga disini emang rata2 segitu. Dan ini foto diambil di jembatan depan cafe ini.',
 'Lokasi yang pas dan view yang kita dapetin itu yang mahal. Disana tersaji suasana Jogja yang selalu dirindukan para wisatawan. Asik untuk nongkrong dan berbagai pilihan cemilan yang bisa dipilih dan yang paling hitss kopi Joss nya (kopi arang)',
 'Tempat enak langsung view tugu jogja kalo malem keren, makanan enak2 banyak pilihan sate2nya.',
 'Lokasi yang pas dan view yang kita dapetin itu yang mahal. Disana tersaji suasana Jogja yang selalu dirindukan para wisatawan. Asik untuk nongkrong dan berbagai pilihan cemilan yang bisa dipilih dan yang paling hitss kopi Joss nya (kopi arang)',
 'Tempat enak langsung view tugu jogja kalo malem keren, makanan enak2 banyak pilihan sate2nya.',
 'Agak lama yah padahal cuma Pesen pecel lele plus teh hangat, rasa sih ok, harga juga ok',
 'Sipppo mantab',

**Many customers express both their satisfaction and dissatisfaction with various aspects of the restaurant. For example, some customers may be satisfied with the taste of the food, but dissatisfied with the facilities or atmosphere of the restaurant. Conversely, there are customers who are satisfied with both the taste and service, but still express dissatisfaction with the price.**

## Review with Positive Sentiment with more than 3 stars

In [89]:
pos_sentiment_more_3_star = df.query('Sentiment > 0.5 and Star > 3')
pos_sentiment_more_3_star.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment
0,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Barep Pratama,Akhirnyaaaa bisa nulis review yang memuaskannn...,5,0.998006
1,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Luqman Heryana,Cafe de paris\nTerletak di timur pantai parang...,5,0.998011
2,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Catharine Winata,"masakannya enak tp kecil2 sesuai dg harganya😊,...",4,0.998822
3,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Dewi Pustaka,"Ownernya baik banget, nginep disini pas Ramadh...",5,0.997596
4,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Kiki Melaendo,"Tempatnya bagus bgt, makanan enak dan cocok bu...",5,0.998603


In [90]:
pos_sentiment_more_3_star.shape

(4240, 6)

In [92]:
pos_sentiment_more_3_star.Review.tolist()[500:540]

['Mantulll endulita',
 'Segar dan murah',
 'Kesini beli mie jawa nyemek, beeehh gurih sekali, enak. Suasananya juga enak.',
 'Tempatnya unik, bakminya enak. Pesen bakmi goreng tapi dibawainnya bakmi godog😂 tapi gapapa',
 'Untuk agan-agan yang suka suasana santai di tengah kota, ini tempat asyik buat nikmati malam santai ditemani kopi yang asyik dan makanan nya itu lho Bakmi Jawa mantap...\nTempatnya luas dan nyaman...\nSo... Tunggu apa lagi??? Langsung aja ke TKP... 😀😀😀',
 'Saya suka ke Bradip karena buka sampai larut, danang lokasi yang masuk tidak terlalu terlihat dari jalan raya.\nCocok untuk meeting kecil ataupun mengobrol dengan teman, karena tidak bising suara kendaraan bermotor.\nMenu yang ditawarkan juga enak, seperti olahan bakmie Jawa dan nasi goreng, ada juga minuman teh poci yang disajikan dengan teko gerabah. Untuk harga affordable dan sesuai dengan rasa menunya.',
 'bisa ngobrol sambil ngopi dan main catur, harga menu sangat bersahabat',
 'Bakmi jawanya mantul bgt dan kop

**The customers tend to express their satisfaction more frequently than the aspects they are dissatisfied with.**

## Review with negative Sentiment with more than 3 stars

In [83]:
neg_sentiment_more_3_star = df.query('Sentiment < 0.5 and Star > 3')
neg_sentiment_more_3_star.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment
7,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Erlina Wahyuni,Melukiskan suasana hati yang sulit dijelaskan,5,0.009842
20,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",steve kanisius,Pemiliknya adalah pasangan anggota Credit Unio...,5,0.035851
24,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Janisda Vria Sandi (Janice),"Good good good,, jalan kaki dari parkiran meni...",5,0.052864
25,ChIJCfxrxdPjei4R5O1EBSra9kc,Aguswijayatech,agus wijaya16,Oke,5,0.007676
57,ChIJ6yf1WiRYei4RZNWve1zo5z4,Anggrek Executive Lounge Stasiun Tugu Yogyakarta,Hello Reina,Sambil nunggu check in hotel karena nyampe sta...,4,0.079342


In [84]:
neg_sentiment_more_3_star.shape

(666, 6)

In [85]:
neg_sentiment_more_3_star.Review.tolist()

['Melukiskan suasana hati yang sulit dijelaskan',
 'Pemiliknya adalah pasangan anggota Credit Union Cindelaras Tumangkar (CUCT). Berlokasi persis di pantai Parangtritis sebelah timur.\n6 Juli 2018 seluruh aktivis CUCT datang ke tempat ini untuk syawalan bersama. Dengan suguhan degan (kelapa muda), nasi, sayur asem, oseng tahu tempe, ikan laut, udang, dan cumi, bakwan, pisang goreng, kacang panggang/bakar.\nDi tempat ini ada ruang pertemuan semi terbuka, dapat memuat lima puluhan orang. Di kanan kiri ruang ini terdapat berjejer-jejer kamar mandi.\nDi depan terdapat tempat nongkrong, gasebo, yang nyaman untuk duduk-duduk bertiga atau berempat, sambil menikmati menu dan melihat bebas ke arah laut Indonesia.\n\nIkan bakar bumbu manisnya enak tenan.',
 'Good good good,, jalan kaki dari parkiran menikmati indahnya pantai, sampai sana laparnya gak ketulungan,pesen makan kondisi anget semua.agk lama sih,tapi sensasinya',
 'Oke',
 'Sambil nunggu check in hotel karena nyampe stasiun jam 3.40 an 

**Many customers provide positive reviews for the restaurant. However, some customers solely offer critiques and express dissatisfaction. Additionally, there are also many customers who provide irrelevant reviews that do not offer any meaningful information about the restaurant.**

## Review with negative Sentiment with 3 stars

In [86]:
neg_sentiment_3_star = df.query('Sentiment < 0.5 and Star == 3')
neg_sentiment_3_star.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment
22,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Nichan,Penjualnya ramah bngt. Harganya jujur sesuai m...,3,0.094220
47,ChIJ6yf1WiRYei4RZNWve1zo5z4,Anggrek Executive Lounge Stasiun Tugu Yogyakarta,ishaan,perorang dikenakan biaya 49ribu cuma dpt minum...,3,0.147827
142,ChIJU-mjYm7jei4RKHXQd_6o15I,Angkringan KlX,Tabita Putri,Orderan di gofood udah 4 kali order tidak diam...,3,0.003291
169,ChIJc_LsfTBYei4R2ZbyfvHGvYw,Angkringan Kopi Joss Tugu Mbak lina,Samudra Rio,"ditambah varian atau menu makanan , kurang...",3,0.009346
194,ChIJk5mlx3Dkei4R6qbkrzjBp5w,Angkringan Kopi Joss,Samudra Rio,"ditambah varian atau menu makanan , kurang...",3,0.009346


In [87]:
neg_sentiment_3_star.shape

(174, 6)

In [88]:
neg_sentiment_3_star.Review.tolist()

['Penjualnya ramah bngt. Harganya jujur sesuai menu. Cuma aksesnya itu, waduh bngtlah. Kl rasa mgkn selera tp sesuai harga saja.',
 'perorang dikenakan biaya 49ribu cuma dpt minum dan 1 roti, minuman bs di pilih teh/ kopi udah itu doang.... ruangan ac nya biasa aja gak dingin... colokan banyak bgt bisa colok hp/ laptop dimanapun..',
 'Orderan di gofood udah 4 kali order tidak diambil, tapi restonya buka :( kalau memang gamau ambil lebih baik di off-kan saja :(',
 'ditambah varian atau menu   makanan ,   kurang nya  tempat parkiran tidak rapi ,   cocok buat kumpul kumpul ..',
 'ditambah varian atau menu   makanan ,   kurang nya  tempat parkiran tidak rapi ,   cocok buat kumpul kumpul ..',
 'Baca referensi ayam madunya katanya paling enak, ternyata jauh dari definisi rasa enak. Sedikit kecewa apalagi nasinya agak dingin dan kaku.',
 'Sayur lodeh point',
 'Ayamnya kecil-kecil, trus geprek tapi gak digeprek sama sambelnya.. jadi nggak berasa masuk ke ayamnya..\n\nTapi ayam en sambelnya ema

**The customers tend to express their dissatisfaction more frequently compared to the aspects they are satisfied with.**

## Review with negative Sentiment less than 3 stars

In [95]:
neg_sentiment_less_3_star = df.query('Sentiment < 0.5 and Star < 3')
neg_sentiment_less_3_star.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment
51,ChIJ6yf1WiRYei4RZNWve1zo5z4,Anggrek Executive Lounge Stasiun Tugu Yogyakarta,Kaharudin Aldian Saputra,"Sangat mengecewakan, memesan tiket Argo Lawu L...",1,0.002359
54,ChIJ6yf1WiRYei4RZNWve1zo5z4,Anggrek Executive Lounge Stasiun Tugu Yogyakarta,d Seprio,"Minumanya ga panas, jahe & teh nya dingin.\nKu...",2,0.047499
56,ChIJ6yf1WiRYei4RZNWve1zo5z4,Anggrek Executive Lounge Stasiun Tugu Yogyakarta,Panji Purnomo,"Agak payah yah, karena after servicenya parah ...",2,0.002633
73,ChIJ25JsWI_jei4Rpmn5VDDs_LU,Angkringan Bandara YIA,myunanda suken,"Harga tidak wajar, esteh + nasi kucing 2 + gor...",1,0.002335
139,ChIJxUnNYSVYei4RWxF9CIWisck,Angkringan Jaman Edan,maghfira diyaana,"Yaampun live music nya super kenceng suaranya,...",1,0.002309


In [96]:
neg_sentiment_less_3_star.shape

(303, 6)

In [97]:
neg_sentiment_less_3_star.Review.tolist()[100:115]

['bbrp kali kemari, untuk makanan minuman rata2 cukup enak dan lengkap. untuk harga standart resto. namun dr bbrp kali kesini pelayanan tidak mengalami perkembangan lebih baik, rata2 pelayan komunikasinya kaku dan cenderung ketus. bbrp kejadin kurang aware dengan konsumen.\nAtmosphere no smokingnya agak cenderung panas (mungkin saatnya service ac) dan musik ambiencenya tifak cocok serta bass nya terlalu berdentum',
 'Saya driver mengantar tamu asing,hujan2 pelayan diam ga nyuruh masuk, biar ga kehujanan',
 'Pengalaman pertama ke sini tidak terlalu baik karena banyak banget nyamuknya sangat mengganggu, saya dan teman saya sampai heran kenapa banyak sekali nyamuk besar2  di dalam ruangan ini. untuk harga sedikit mahal tapi minumannya enak.\nPilihan menu sedikit.\nPelayanan bagian kasir buruk bgt, karena pas pesen minum minta panas pas di anter kok dingin pakai es, ya udahlah mungkin karena hujan deres jadi mas yg di kasir ga denger saya ngomong minumnya panas.\nTapi yg lebih parah lagi p

**The customers express their dissatisfaction with the restaurant or establishment in a clear manner.**

# Evaluating Model

* True Positive : Review with Positive Sentiment with 3 stars or more 
* True Negative : Review with Negative Sentiment with 2 stars or less
* False Postive : Review with Positive Sentiment with 2 stars or less
* False Negative : Review with Negative Sentiment with 4 stars or more

In [99]:
TP = df.query('Sentiment > 0.5 and Star >= 3').shape[0]
TN = df.query('Sentiment < 0.5 and Star < 3').shape[0]
FP = df.query('Sentiment > 0.5 and Star < 3').shape[0]
FN = df.query('Sentiment < 0.5 and Star >= 3').shape[0]

In [100]:
Recall = TP / (TP + FN)
Precision = TP / (TP + FP)
Accuracy = (TP + TN) / (TP+TN+FP+FN)
F1_measure = (2*Recall*Precision) / (Recall + Precision)

In [101]:
print('The Model Evaluation Metrics : ')
print(f"The recall is {Recall:.2f}")
print(f"The precision is {Precision:.2f}")
print(f"The accuracy is {Accuracy:.2f}")
print(f"The F1 measure is {F1_measure:.2f}")

The Model Evaluation Metrics : 
The recall is 0.84
The precision is 0.99
The accuracy is 0.84
The F1 measure is 0.91


# Merged the Cleaned Reviews with positive sentiment and more than 3 stars rating

In [103]:
df['cleaned_reviews'] = cleaned_reviews.tolist()

In [104]:
df.head()

,Place_ID,Place_Name,User,Review,Star,Sentiment,cleaned_reviews
0,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Barep Pratama,Akhirnyaaaa bisa nulis review yang memuaskannn...,5,0.998006,akhir bisa menulis review yang memuaskan lagi ...
1,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Luqman Heryana,Cafe de paris\nTerletak di timur pantai parang...,5,0.998011,cafe deh paris terletak di timur pantai parang...
2,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Catharine Winata,"masakannya enak tp kecil2 sesuai dg harganya😊,...",4,0.998822,masakan enak tetapi kecil sesuai dengan harga ...
3,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Dewi Pustaka,"Ownernya baik banget, nginep disini pas Ramadh...",5,0.997596,owner baik banget menginap di sini pas ramadan...
4,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",Kiki Melaendo,"Tempatnya bagus bgt, makanan enak dan cocok bu...",5,0.998603,tempat bagus banget makanan enak dan cocok bua...


In [105]:
df_selected = df[['Place_ID', 'Place_Name','cleaned_reviews', 'Star', 'Sentiment']]
df_selected.head()

,Place_ID,Place_Name,cleaned_reviews,Star,Sentiment
0,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",akhir bisa menulis review yang memuaskan lagi ...,5,0.998006
1,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",cafe deh paris terletak di timur pantai parang...,5,0.998011
2,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",masakan enak tetapi kecil sesuai dengan harga ...,4,0.998822
3,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",owner baik banget menginap di sini pas ramadan...,5,0.997596
4,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",tempat bagus banget makanan enak dan cocok bua...,5,0.998603


In [108]:
df_selected = df_selected.query('Star > 3 and Sentiment > 0.5')[['Place_ID', 'Place_Name', 'cleaned_reviews']]
df_selected.head()

,Place_ID,Place_Name,cleaned_reviews
0,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",akhir bisa menulis review yang memuaskan lagi ...
1,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",cafe deh paris terletak di timur pantai parang...
2,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",masakan enak tetapi kecil sesuai dengan harga ...
3,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",owner baik banget menginap di sini pas ramadan...
4,ChIJU39AYwyqey4RXFIG9IudN8M,""" Cafe de Paris "" Home Stay & Resto Seafood",tempat bagus banget makanan enak dan cocok bua...


In [112]:
len(df_selected.Place_ID.unique())

357

In [109]:
df_fix = df_selected.groupby(['Place_ID', 'Place_Name']).agg({'cleaned_reviews': ' '.join}).reset_index()

In [115]:
df_fix.query('Place_ID == "ChIJU39AYwyqey4RXFIG9IudN8M"').cleaned_reviews.tolist()

['akhir bisa menulis review yang memuaskan lagi enak banget ikan bakarnyaa makanan semua oke sentuhan bumbu lain dari pada yang lain ibu bapak lupa belum kenalan super ramah couple goals bisa haha hihi sampai menua bersama sehat selalu bapak ibu janji deh akan balik lagi suka banget sama tempat dan masakan ibu bule saja makan di sini wisatawan lokal jangan mau kalah dongggjelajahi terus wisata indonesia terima kasih bonus tuna busempat galau antara kakap atau tuna akhir malah bisa merasakan keduanyaa cafe deh paris terletak di timur pantai parangtritis dan akses dengan jalan kaki dari parkiran terakhir makanan cukup banyak pilihan mulai dari seafood sayur dan minuman suasan nyaman dan santai dengan banyak pilihan tempat duduk ayunan tempat tidur gantung serta gazebo konsep nya cafe pantai yang asyik harga nya murah untuk di tempat wisata worth it masakan enak tetapi kecil sesuai dengan harga pisang goreng mantap sambel mantap owner baik banget menginap di sini pas ramadan pesan makanan

In [118]:
df_fix.to_csv('/content/drive/MyDrive/Capstone | Bangkit 2023/Sentiment Analysis/aggregated_reviews.csv')